In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import requests

In [2]:
# Set random seed for reproducibility
np.random.seed(42)

# CONVERSION ABC

In [3]:
# Set random seed for reproducibility
np.random.seed(42)

# Number of rows in the dataset
num_rows = 1000

# Generate random activation dates
start_date = datetime(2022, 1, 1)
end_date = datetime(2022, 12, 31)

# Generate data for IsConversion column with a normal distribution for each group
is_conversion_probabilities = {
    'Group_A': np.random.normal(loc=0.4, scale=0.2, size=num_rows),
    'Group_B': np.random.normal(loc=0.5, scale=0.2, size=num_rows),
    'Group_C': np.random.normal(loc=0.7, scale=0.2, size=num_rows)
}

is_conversion = np.concatenate([
    (probabilities > 0.5).astype(int) for probabilities in is_conversion_probabilities.values()
])

# Generate random activation dates for each group
activation_dates = np.concatenate([
    [np.random.choice(pd.date_range(start_date, end_date)) for _ in range(num_rows)]
    for _ in range(len(is_conversion_probabilities))
])

# Generate random group names
groups = np.concatenate([
    [group] * num_rows for group in is_conversion_probabilities.keys()
])

# Create DataFrame
df = pd.DataFrame({
    'IsConversion': is_conversion,
    'ActivationDate': activation_dates,
    'Group': groups
})

# Sort DataFrame by ActivationDate and Group
df = df.sort_values(by=['Group', 'ActivationDate']).reset_index(drop=True)

# Generate random conversion dates based on activation dates for each group
df['ConversionDate'] = df.groupby('Group')['ActivationDate'].transform(lambda x: x + pd.to_timedelta(np.random.randint(1, 30, size=len(x)), unit='D'))

# Update ConversionDate to null when IsConversion is equal to zero
df.loc[df['IsConversion'] == 0, 'ConversionDate'] = np.nan

# Write DataFrame to CSV
df.to_csv('conversionABC.csv', index=False)

# Display the DataFrame
print(df.head())


   IsConversion ActivationDate    Group ConversionDate
0             0     2022-01-01  Group_A            NaT
1             0     2022-01-01  Group_A            NaT
2             1     2022-01-01  Group_A     2022-01-22
3             0     2022-01-01  Group_A            NaT
4             0     2022-01-03  Group_A            NaT


# statcan New Motor Vehicle Sales

In [4]:
def download_csv(url, destination):
    response = requests.get(url)
    with open(destination, 'wb') as file:
        file.write(response.content)

csv_url = 'https://www150.statcan.gc.ca/t1/tbl1/en/dtl!downloadDbLoadingData-nonTraduit.action?pid=2010000101&latestN=0&startDate=20190101&endDate=20231101&csvLocale=en&selectedMembers=%5B%5B1%5D%2C%5B1%5D%2C%5B1%5D%2C%5B1%2C2%5D%2C%5B1%5D%5D&checkedLevels='
destination_path = 'statcan_NewMotorVehicleSales.csv'

download_csv(csv_url, destination_path)

In [5]:
df = pd.read_csv(destination_path, encoding='utf-8')
df.head(5)

,REF_DATE,GEO,DGUID,Vehicle type,Origin of manufacture,Sales,Seasonal adjustment,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2019-01,Canada,2016A000011124,"Total, new motor vehicles","Total, country of manufacture",Units,Unadjusted,Units,300,units,0,v42169911,1.1.1.1.1,112178,NaN,NaN,NaN,0
1,2019-02,Canada,2016A000011124,"Total, new motor vehicles","Total, country of manufacture",Units,Unadjusted,Units,300,units,0,v42169911,1.1.1.1.1,124493,NaN,NaN,NaN,0
2,2019-03,Canada,2016A000011124,"Total, new motor vehicles","Total, country of manufacture",Units,Unadjusted,Units,300,units,0,v42169911,1.1.1.1.1,187907,NaN,NaN,NaN,0
3,2019-04,Canada,2016A000011124,"Total, new motor vehicles","Total, country of manufacture",Units,Unadjusted,Units,300,units,0,v42169911,1.1.1.1.1,186487,NaN,NaN,NaN,0
4,2019-05,Canada,2016A000011124,"Total, new motor vehicles","Total, country of manufacture",Units,Unadjusted,Units,300,units,0,v42169911,1.1.1.1.1,208375,NaN,NaN,NaN,0
